# EXtra-foam azimuthal integration benchmark

In [ ]:
import numpy as np
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator as PyfaiAzimuthalIntegrator
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

import extra_foam
print(extra_foam.__version__)

from extra_foam.algorithms.azimuthal_integ import AzimuthalIntegrator
from extra_foam.algorithms import mask_image_data

In [ ]:
img = np.load("./jf_ring.npy")
mask = np.zeros_like(img, dtype=bool)
mask_image_data(img, out=mask)
_, ax = plt.subplots(figsize=(8, 12))
ax.imshow(img)

In [ ]:
dist = 1  # sample distance
npt = 512  # number of integration points
pixel1, pixel2 = 0.75e-6, 0.75e-6  # pixel size (y, x)
poni1, poni2 = -34 * pixel1, 1106 * pixel2  # integration center (y, x)

In [ ]:
pyfai_integrator = PyfaiAzimuthalIntegrator(
    dist=dist, poni1=poni1, poni2=poni2, pixel1=pixel1, pixel2=pixel2, wavelength=1e-10)

%timeit pyfai_integrator.integrate1d(img, npt, mask=mask, unit="q_A^-1")
q_gt, I_gt = pyfai_integrator.integrate1d(img, npt, mask=mask, unit="q_A^-1")

In [ ]:
integrator = AzimuthalIntegrator(
    dist=dist, poni1=poni1, poni2=poni2, pixel1=pixel1, pixel2=pixel2, wavelength=1e-10)

%timeit integrator.integrate1d(img, npt=npt)
q, I = integrator.integrate1d(img, npt=npt)

In [ ]:
_, ax = plt.subplots(figsize=(12, 6))

ax.plot(1e-10 * q, I, label='EXtra-foam')
ax.plot(q_gt, I_gt, label='pyFAI')
ax.set_xlabel("q (1/A)", fontsize=16)
ax.set_ylabel("I (arb.)", fontsize=16)
ax.legend(fontsize=16)